:language_code: pt_br

`pirp_4e_aux:result_3_gg Resultados` | `pirp_4e_aux:activity_user_username_3_gg Modelo submissão aluno`

=======================================================
pirp_4e_gg - iaffine, modificação pixels de fora
=======================================================

Criando o teste
===============

.. code:: python module
    
    from tbx_perftest.tester import Tester
    
    def assert_fnc(user_fnc, input, groundtruth):
        f, T = input
        return float(abs(user_fnc(f,T) - groundtruth).max() <= 1.)
    
    tester = Tester(assert_fnc, 'tbx_perftest', 'pirp_4e_gg',runs=10)

=======================================================================================
Criação dos casos de teste e de treino juntamente com pesos para formação da nota final
=======================================================================================

.. code:: python
    
    from tbx_perftest.pirp_4e_gg import tester
    import numpy as np
    import ia636 as ia
    
    # IMPORTANTE: Os casos de teste devem ser apagados se esta página sofrer múltiplas edições
    tester.reset_cases()
    
    def gg_gabarito(f,T):
       
       shapef = f.shape   
       g = np.zeros_like(f)
       H,W = f.shape;
       y1,x1 = np.indices((H,W))
       p1 = np.array([ y1.ravel(), 
                       x1.ravel(), 
                       np.ones(H*W)]) # Preparing the augmented indexes matrix.  
       p = np.dot(np.linalg.inv(T), p1) # Performing the inverse mapping
        
       # Using the Nearest Neighbor interpolation method.
       y = np.rint(p[0]).astype(int).reshape(H,W)
       x = np.rint(p[1]).astype(int).reshape(H,W)
       mask = (y>=0) & (y<H) & (x>=0) & (x<W)
       g[mask] = f[y[mask],x[mask]]
       return g
       
Caso 0

.. code:: python    
    
    a = np.resize(np.arange(24),(4,6))
    Ts   = np.array([[ 1.0, 0.0,  2.0], 
                     [ 0.0, 1.0, -1.0], 
                     [ 0.0, 0.0,  1.0]])
    g = ia.iaffine(a,Ts) 
    print a
    print g
    g1 = gg_gabarito(a,Ts)
    print g1

    tester.add_test_case([a,Ts], gg_gabarito(a,Ts), label='numerico 0', show_in_results_table=True)
    tester.add_train_case([a,Ts], gg_gabarito(a,Ts))

Caso 1

.. code:: python

    a = np.resize(np.arange(24),(4,6))
    Ts   = np.array([[ 0.5, 0.0, 0.0], 
                     [ 0.0, 1.5, 0.0], 
                     [ 0.0, 0.0, 1.0]])
    g = ia.iaffine(a,Ts) 
    print a
    print g
    g1 = gg_gabarito(a,Ts)
    print g1
    
    tester.add_test_case([a,Ts], gg_gabarito(a,Ts), label='numerico 1', show_in_results_table=True)
    tester.add_train_case([a,Ts], gg_gabarito(a,Ts))

Caso 2

.. code:: python
    
    f = adreadgray('cameraman.tif')
    theta = pi/4
    scale = [0.5,0.5]
    t = [0,64]
    
    Ts   = [[scale[1],0,0], [0,scale[0],0], [0,0,1]]                         #Scale
    Trot = [[cos(theta),-sin(theta),0], [sin(theta),cos(theta),0], [0,0,1]]  #Rotation
    Tx   = [[1,0,t[1]], [0,1,t[0]], [0,0,1]]                                 #Translation
    T = dot(Tx,dot(Ts,Trot))
    g = ia.iaffine(f,T)
    adshow(f,"Original image")
    adshow(g,"Affine transformation")
    g1 = gg_gabarito(f,dot(Tx,dot(Ts,Trot)))
    adshow(g1,"Affine transformation")

    tester.add_test_case([f,T], gg_gabarito(f,T), label='image 2', show_in_results_table=True)
    tester.add_train_case([f,T], gg_gabarito(f,T))
    

Entrada e Saída
===============

.. code:: python

       import numpy as np
       import ia636 as ia
       
       f0, T0 = tester.get_input(0)
       g0 = tester.get_output(0)
       #print 'input:', tester.get_input(0)
       #print 'output:\n', tester.get_output(0)

       f1, T1 = tester.get_input(1)
       g1 = tester.get_output(1)

       f2, T2 = tester.get_input(2)
       g2 = tester.get_output(2)
       print 'T2=\n', T2
       adshow(f2, 'entrada')
       adshow(g2, 'saída')

Gabarito
========

.. code:: python

       print 'gabarito=\n',gg_gabarito(a,Ts)       
       #adshow(ia.ianormalize(gg_gabarito(257)),'Imagem gabarito')
